# Collective2 Data for quantitative analysis - options

## Content

- [Using SQL](#Using-just-SQL)
- [Using Node.js](#Using-Node.js)
- [Using C\#](#Using-C#)
- [Using R](#Using-R)
- [REST API](#REST-API)
- [GraphQL](#GraphQL)  

- When we talk about APIs, we mean programmers and data. 
- When we talk about programmers, we mean programming languages.
- When we talk about data, we mean databases.
- When we talk about a quantitative analysis, we mean lots and lots and lots of data (ideally ALL available data).

Our data lives in a database, and we're going to create an API that allows programmers to access the data in the database for quantitative analysis.

So we need to implement an API server, which is a middle-ware between a database(s) programmers.  API server must ensure not only data delivery, but also ensure that only publicly visible data are delivered. I am not talking about authentication/authorization here, because I suppose the API will be just read-only. 

An API server is a peace of the software which:
- Uses its own, original "language" (specification) which must be documented and learned.
- It implements its own, original functions set, which limits users' imagination.
- It creates a "resistance point" that slows down the transmission of large amounts of data.  

Anyway, the very first point of the API implementation is a database.

What if we "commoditize" this first, neccessary step?

What if we create a publicly available database containing clean, publicly visible data and give access to it to the programmers which will use their programming languages to create functions they need for quantitative analysis?
 
Database access has following advantages above an API:
- It uses well known, powerful SQL language.
- The SQL language contains many functions too. But such functions do not limit users imagination and data usage.
- Data transfer is fast.
- Database can be dumped and stored locally which creates even much faster and a stable source of data for investigation. When done on the local instance, it is easy to switch to the remote server.
- It is much easier to work with SQL using IDE like MySQL Workbench, than with API.
- Data can be easily exported to CSV and used i.e. in Excel (like Daniil) or in other software. 

So I believe, that if we start with a separate, publicly accessible database running in a secure environment (i.e. in Docker), we can't make a mistake.

We can build an API on the top of such database then.

The separate database has also advantages on our side:
- Running on the separate machine, it will not add load to our production MySql server
- When using APIs, it is usually necessary to clear and select publicly visible data each time the API is called. However, such a selection can only be done once, and that is once per night when the database is updated. It does not then increase the load on our production servers.
- If someone decide to perform a DoS attack, it does not affect our production server. We can set limits in the server settings and we do not need to implement rate limits or something like that in each API endpoint.
- Perhaps we will find that the database full of data is enough for our users. Then we would completely save the cost of API implementation. :-)

Good, clean data is the basis for people to analyze it. We should deliver the best quality data we are able to produce. We have no idea if people will use Excel, technical analysis, charts, statistics, artificial intelligence... But we do know that people have these tools and that they know how to use them. But they need good data and they need it quickly and easily.

See the following Jupyter Notebooks for examples how to use MySql in different languages.

Last one is about GraphQL API.


---
### Daniil's Score Workbench formula

We will use this Daniil's Score Workbench formula as an example in this text.

~~~~perl
#Risk metrics 
(
(20.0 - %[Max Equity Drawdown (Percent)]% * (20.0/45))
+
(%[Max Open Loss Percent 90 Days]% == "-" ? 0 : 20.0 - %[Max Open Loss Percent 90 Days]% * (20.0/8))
+
(10 - %[Percent Trades Options]% * (10/1))
+
(%[Short Options - Percent Covered]% >= 100 ? 10 : %[Short Options - Percent Covered]% / 100 * 10)


#Behavioral metrics 
+
(%[Age Strategy (Days)]% >= 720 ? 40.0 : %[Age Strategy (Days)]%/720 * 40.0)
+
(20.0 - %[Daily Max Leverage (Max)]% * (20.0/10))
+
(%[Latest Trade Days Ago]% <= 60 ? 20 : 0) 
+
(%[Percentage Months Profitable]% >= 100 ? 5 : %[Percentage Months Profitable]%/100 * 5)
+
(%[Num Trades All]% >= 100 ? 20 : %[Num Trades All]%/100 * 20)
+
((%[Delta Equity 90 Days]% <= 0 and %[Delta Equity 90 Days]% > -7) ? 5 : 0)


#Profitability metrics 
+
(%[Annualized Return Pcnt]%  > 0.6 ? 20 : %[Annualized Return Pcnt]% / 0.6 * 20)
)

#Exclude strategies
*
(%[Age Strategy (Days)]% < 90 ? 0 : 1)
*
(%[Annualized Return Pcnt]% < 0 ? 0 : 1)
*
(%[Delta Equity 180 Days]% < 0 ? 0 : 1)

#score_greater_than: 0#
~~~~

### Using SQL


A notebook which shows how to replicate Collective2 Scoring Workbench functionality using just SQL.

[See this notebook.](./Using_SQL.ipynb)

### Using Node.js

How to use data in Node.js.

- [This notebook](./Daniil_Score_JS.ipynb) shows Daniil's formula implemented in JavaScript/TypeScript running in Node.js.

- [This notebook](./BollingerBands_TS.ipynb) is an example how to plot Bollinger bands for the equity.

### Using C\#

[This notebook](./CSharpWindowsApp.ipynb) is an example how to visualize database data in the Windows application.

### Using R

This project was implemeted in R. It uses a local MySql database: [Collective2 StrategyStats](https://github.com/collective2/StrategiesStats/blob/main/START_HERE.ipynb)

### REST API

If a separate database is not a way you want to go, we can implement a REST API and try to deliver data from some of our projects. I think the following are good candidates:
- The Grid 
- Scoring Workbench
- My Collective2 StrategyStats project data

Perhaps from Firehose, Smart portfolio, NetAutotrade positions, Portfolio Time Machine then.

### GraphQL

I looked at [GraphQL API](https://graphql.org/).

It is certainly more advanced and flexible then a REST API.

If I understand well, there are two main problems: 
- Data should be in memory to make it work quickly.
- The server implementation is probably quite labor intensive.

I have found a project which solves those problems: [Hasura](https://hasura.io/) 

But it does not work with MySql yet. [MySQL preview](https://hasura.io/docs/latest/graphql/core/guides/mysql-preview.html#id1)

So we would need a separate PostgreSQL database updated daily. It should not be a big deal.

GraphQL has some SQL features like projection, selection and joining. It even enables to integrate several different datasources in one query. For example our database and our API V3 and API V4 in one query.

I tried to use it, but it would need more time. It is a new thing which needs more learning. 


### GraphQL query example

This is not a real-world usage query. Just an example what is possible. It is quite impressive.

It returns maxima of several data fileds (API calculates them!) and also selected data of all strategies in one API call. 

![Image](./Hasura_MaxQuery.png)


However, a similar thing can easily be done by anyone with access to the underlying database :-)